In [145]:
from numpy import array
import numpy
import random
from deap import base, creator, tools, algorithms
import pandas  as pd
import warnings
import tqdm as tqdm
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

###  Multi-objective Optimization Problem  ###

df_selected = pd.read_csv("df_selected.csv").drop("Unnamed: 0",axis=1)

df_selected.head(1)

,S_06_MUNICIPIO,S_10_NOME_ENTREVISTADO,S_11_SEXO,S_15_IDADE,S_21_AREA_PROPRIEDADE,S_23_CASO_POSITIVO_QUANTOS_KVA,S_53_TEM_PONTO_DE_ÁGUA,S_25_E_COOPERADO_SARONCRED,S_26_CASO_NAO_COOPERADO_MOTIVO,S_27_E_COOPERADO_A_APROCAN,S_28_CASO_NAO_ASSOCIADO_MOTIVO,S_29_SITUACAO_CADASTRAL_IMA,S_50_PRODUÇÃO_TOTAL_DE_LEITE_DIA,S_55_POSSUI_IMPLANTADA_AS_BPF,S_56_POSSUI_TREINAMENTO_EM_BPF,S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO,S_77_2_QUANTIDADE_QUEIJO_CANASTRA,S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA,S_93_POSSUI_IMPLANTADA_BPF,S_94_POSSUI_TREINAMENTO_EM_BPF,S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO,S_96_COM_QUEM_APRENDEU,S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR,S_130_QUAL_RENDA_MENSAL_PRODUÇAO,S_132_1_PESSOAS_FAMILIA_ATÉ_12_ANOS,S_132_2_PESSOAS_FAMILIA_ATÉ_18_ANOS,S_132_3_PESSOAS_FAMILIA_ADULTOS_ATÉ_65_ANOS,S_132_4_PESSOAS_FAMILIA_ACIMA_DE_65_ANOS,S_134_FAZ_USO_CRÉDITO_RURAL,S_135_PARA_QUE_FAZ_USO_CRÉDITO_RURAL,S_136_FONTE_DE_RECURSO,S_137_MOTIVO_PARA_NÃO_USAR_CRÉDITO_RURAL,S_139_PROPRIEDADE_FAZ_CONTROLE_FINANCEIRO,S_140_COMO_FAZ_CONTROLE_FINANCEIRO,S_141_PROPRIEDADE_FAZ_ALGUM_TIPO_PREVISAO,S_145_FAZ_USO_ASSISTENCIA_TECNICA,S_146_PORQUE_NAO_FAZ_USO_DE_ASSISTENCIA_TECNICA,S_147_TIPO_MAO_DE_OBRA_UTILIZADA,S_148_NUMERO_FAMILIARES_TRABALHAM_PROPRIEDADE,S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM
0,SÇO ROQUE DE MINAS,JOSE FLORENCIO DA SILVA,MASCULINO,77.0,1452.0,3,SIM,NAO,POUCO CONHECIMENTO,NAO,NAO PERCEBI VANTAGENS,RELACIONADO,330.0,NAO,NAO,50.0,42,9.0,NAO,NAO,28.0,FAMILIARES,10000.0,ACIMA DE 10 ATE 20 SALARIOS MINIMOS,2.0,1.0,3,2.0,NÇO,NAO SABE,OUTROS,OUTROS,NAO,ANOTA€OES EM BLOCOS E OU CADERNOS,NAO,FIRMA PARTICULAR,NUNCA PROCUREI,CONTRATADA,0,3.0


In [150]:
param_select=[
    
    #K1
    "S_21_AREA_PROPRIEDADE",
    "S_23_CASO_POSITIVO_QUANTOS_KVA",              
    #Água  - Feature engineered
    
    #K2
    
    #Soro de leite  - Feature engineered
    #Solid Waste
    
    #K3
    
    "S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO",
    "S_77_2_QUANTIDADE_QUEIJO_CANASTRA",
    "S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA",
    "S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR",
    
    #K4
    
    "S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM",
    #Crédito Rural
    
    #K5    
    
    #Tax - Feature engineered    
    "S_15_IDADE",
    "S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO"
    ]

df_main = df_selected[param_select].astype("int16")

In [149]:
#S_15_IDADE  S_21_AREA_PROPRIEDADE   S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM
df_main.head(1)

,S_21_AREA_PROPRIEDADE,S_23_CASO_POSITIVO_QUANTOS_KVA,S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO,S_77_2_QUANTIDADE_QUEIJO_CANASTRA,S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA,S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR,S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM,S_15_IDADE,S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO
0,1452,3,50,42,9,10000,3,77,28


In [155]:
IND_INIT_SIZE = 10

#K1

MAX_AREA = df_main["S_21_AREA_PROPRIEDADE"].values.max() 
MAX_ELETR = df_main["S_23_CASO_POSITIVO_QUANTOS_KVA"].values.max() 

#K2


#Soro de leite  - Feature engineered
#Solid Waste


#K3

MAX_COW = df_main["S_67_NÚMERO_TOTAL_REBANHO_LEITEIRO"].values.max() 
MAX_KG = df_main["S_77_2_QUANTIDADE_QUEIJO_CANASTRA"].values.max() 
MAX_PRECO = df_main["S_79_2_PREÇO_MÉDIO_RECEBIDO_QUEIJO_CANASTRA"].values.max() 
MAX_RECEITA = df_main["S_129_QUAL_RENDA_TOTAL_MENSAL_FAMILIAR"].values.max() 

#K4

MAX_CONTRATACAO = df_main["S_149_QUANTAS_PESSOAS_CONTRATADAS_TRABALHAM"].values.max() 
#Crédito Rural
#APOIO DA APROCAN - EM TERMOS DE HORAS

#K5

#Tax - Feature engineered    
MAX_IDADE = df_main["S_15_IDADE"].values.max()
MAX_TEMPO = df_main["S_95_HÁ_QUANTO_TEMPO_PRODUZ_QUEIJO"].values.max()

#Importar pandas e converter em lista de NUMPY

# Create the item dictionary:

r = df_main.values

# r = array([[213, 508,  22],  # 1st arg : weight / 2nd arg : size / 3rd arg : value
#            [594, 354,  50],
#            [275, 787,  43],
#            [652, 218,  46],
#            [728, 183,  43],
#            [856, 308,  33],
#            [727, 482,  45],
#            [762, 683,  26],
#            [707, 450,  19],
#            [899, 825,  17],
#            [446, 360,  35],
#            [491, 818,  47],
#            [647, 404,  17],
#            [604, 623,  32],
#            [900, 840,  45],
#            [374, 127,  33]] )

In [153]:
r.shape

(757, 9)

In [154]:
NBR_ITEMS = r.shape[0]

items = {}
# Create random items and store them in the items' dictionary.

for i in range(NBR_ITEMS):
    items[i] = (r[i][0] , r[i][1] , r[i][2] , r[i][3] , r[i][4] , r[i][5] , r[i][6], r[i][7], r[i][8])

creator.create("Fitness", base.Fitness, weights=(-1,1))  # Note here <- I used only two weights!  (at first, I tried weights=(-1.0 , -1.0, 1.0)) but it crashes. With deap, you cannot do such a thing.

creator.create("Individual", set, fitness=creator.Fitness)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, n=IND_INIT_SIZE) #
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [158]:
def evaluation(individual):
    area = 0
    kva = 0
    rebanho = 0
    queijo = 0
    preco = 0
    renda = 0
    pessoas = 0 
    idade = 0
    tempo = 0
    

    # Maximize or Minimize Conditions
    for item in individual:
        area += items[item][0]  # It must be maximized.
        kva += items[item][1]  # It must be maximized.
        rebanho += items[item][2]  # It must be maximized.
        queijo += items[item][3]  # It must be maximized.
        preco += items[item][4]  # It must be maximized.
        renda += items[item][5]  # It must be maximized.
        pessoas += items[item][6]  # It must be maximized.
        idade += items[item][7]  # It must be minimized.
        tempo += items[item][8]  # It must be maximized.
    
    # Limit Conditions
    if area > MAX_AREA:
        area = 2500
    
    if kva > MAX_ELETR:
        kva= 3

    if rebanho > MAX_COW:
        rebanho= 10
        
    if queijo > MAX_KG:
        queijo= 10
        
    if preco > MAX_PRECO:
        preco= 8
        
    if renda > MAX_RECEITA:
        renda= 100
        
    if pessoas > MAX_CONTRATACAO:
        pessoas= 2
        
    if idade > MAX_IDADE:
        idade= 50
        
    if tempo > MAX_TEMPO:
        tempo= 10

    MinFitess_score = idade   # NOTE : Minimize weight, size
    MaxFitenss_score = area + kva + rebanho + queijo + preco + renda +  pessoas + tempo # NOTE : Maximize weight, size
    
    return MinFitess_score , MaxFitenss_score


def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)  # Used in order to keep type
    ind1 &= ind2  # Intersection (inplace)
    ind2 ^= temp  # Symmetric Difference (inplace)
    return ind1, ind2


def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:  # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,  # NOTE comma(,) , if there's no comma, an error occurs.



toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2) # NSGA-2 applies to multi-objective problems such as knapsack problem
toolbox.register("evaluate", evaluation)


def main():
    ngen = 1000  # a number of generation  < adjustable value >

    pop = toolbox.population(n= 300)
    hof = tools.ParetoFront() # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, ngen=ngen, stats=stats, halloffame=hof, verbose=True)

    return hof, pop

if __name__ == "__main__":
    hof, pop = main()

    print(hof) # non-dominated individuals' list  # the fittest value is placed on the most right side.

gen	nevals	avg                          	std                          	min        	max            
0  	300   	[  50.         2091.81666667]	[   0.         3404.75799283]	[ 50. 512.]	[   50. 21768.]
1  	229   	[  36.41666667 1678.9       ]	[  22.66531246 3043.5695069 ]	[    -0. -26527.]	[   78. 22415.]
2  	224   	[  29.24 1689.14]            	[  25.94575881 3119.75110232]	[    -0. -26527.]	[   92. 18914.]
3  	226   	[  25.20333333 2060.78333333]	[  27.15514664 3695.79950526]	[    -0. -11290.]	[   92. 18914.]
4  	233   	[  19.26333333 1511.92666667]	[  25.06153737 3110.57083099]	[    -0. -11290.]	[   92. 18456.]
5  	249   	[  17.58333333 1738.94      ]	[  25.14298555 4160.91299714]	[    -0. -10762.]	[   90. 20426.]
6  	211   	[  17.59666667 1497.03666667]	[  25.85073801 3617.21867305]	[-0.  0.]        	[   92. 20426.]
7  	219   	[  16.53333333 1574.02333333]	[  24.70294629 3707.63152378]	[-0.  0.]        	[   91. 20440.]
8  	215   	[  15.94666667 1400.63      ]	[  25.30198587 3467.183230

81 	233   	[  13.82666667 1470.73333333]	[  23.7744251  3767.64141193]	[-0.  0.]        	[   91. 20288.]
82 	224   	[  12.41333333 1396.34333333]	[  22.5579806  3693.65744019]	[-0.  0.]        	[   90. 20416.]
83 	235   	[ 10.25666667 960.72666667]  	[  20.9689641  3016.36216083]	[-0.  0.]        	[   88. 19679.]
84 	234   	[ 11.04666667 988.84      ]  	[  21.33427185 2917.38004399]	[-0.  0.]        	[   88. 20081.]
85 	223   	[  11.47333333 1016.3       ]	[  22.87114533 2909.88147009]	[-0.  0.]        	[   92. 20259.]
86 	227   	[  11.62666667 1206.21333333]	[  22.00107472 3512.61567323]	[-0.  0.]        	[   83. 20805.]
87 	256   	[ 10.33333333 885.58333333]  	[  20.68031807 2647.30298412]	[-0.  0.]        	[   73. 21123.]
88 	205   	[ 11.24333333 944.06666667]  	[  21.53982642 3230.94711123]	[    -0. -26527.]	[   91. 21123.]
89 	212   	[ 11.19666667 990.41      ]  	[  21.80209445 3296.19295682]	[    -0. -24581.]	[   91. 18513.]
90 	217   	[  12.60333333 1192.75666667]	[  23.62440805

167	236   	[  13.63 1331.8 ]            	[  24.18290925 3168.30831202]	[-0.  0.]        	[   91. 17072.]
168	235   	[  12.56666667 1306.46666667]	[  23.52584867 3523.10656035]	[-0.  0.]        	[   91. 20313.]
169	237   	[  13.53 1285.89]            	[  23.54745068 4025.36380938]	[    -0. -30975.]	[   88. 20311.]
170	242   	[  11.93666667 1373.41333333]	[  22.30394559 3712.54859575]	[-0.  0.]        	[   90. 20311.]
171	232   	[  12.70666667 1331.51333333]	[  22.90008637 3587.18276133]	[-0.  0.]        	[   92. 20866.]
172	218   	[  14.88 1413.6 ]            	[  24.36689558 4070.17022822]	[    -0. -26527.]	[   88. 19286.]
173	225   	[  13.32 1267.47]            	[  24.21248163 3996.47676616]	[    -0. -25745.]	[   87. 20639.]
174	260   	[  10.5 1268.3]              	[  21.53655807 4177.21436247]	[    -0. -25745.]	[   91. 21645.]
175	229   	[  11.57666667 1096.94666667]	[  22.96934455 3826.50266743]	[    -0. -25745.]	[   91. 20892.]
176	224   	[  12.22333333 1020.68666667]	[  23.22441507

248	223   	[  13.67666667 1301.06666667]	[  23.17913118 3901.14047968]	[    -0. -27724.]	[   85. 19861.]
249	229   	[  11.56666667 1191.        ]	[  22.66845581 3596.28100589]	[    -0. -14045.]	[   89. 20014.]
250	226   	[  10.81666667 1045.61      ]	[  20.53183842 2955.40401376]	[-0.  0.]        	[   70. 20014.]
251	218   	[  12.34       1077.91333333]	[  22.23490649 2891.04928342]	[-0.  0.]        	[   82. 18093.]
252	234   	[  13.22333333 1293.98666667]	[  23.55518603 3258.12138916]	[-0.  0.]        	[   88. 19568.]
253	211   	[  13.46666667 1409.37333333]	[  23.68914426 3398.43638074]	[-0.  0.]        	[   88. 20053.]
254	233   	[  14.07       1577.04666667]	[  23.99204938 3742.22124295]	[-0.  0.]        	[   88. 20230.]
255	218   	[  14.37666667 1636.47333333]	[  23.82466766 3890.81197129]	[-0.  0.]        	[   88. 19208.]
256	230   	[  11.58666667 1461.24      ]	[  21.78154775 3829.67302726]	[-0.  0.]        	[   84. 19841.]
257	227   	[  13.21 1444.19]            	[  23.32536316

334	211   	[  12.84333333 1427.21333333]	[  22.70122146 3707.55508224]	[-0.  0.]        	[   90. 21284.]
335	238   	[  12.52666667 1122.06666667]	[  22.38710839 3626.63978575]	[    -0. -27510.]	[   88. 19321.]
336	226   	[  11.61666667 1003.88333333]	[  22.77036939 2997.16808611]	[   -0. -5196.]  	[   91. 20097.]
337	225   	[   9.55333333 1069.82      ]	[  19.97165881 3294.06327721]	[-0.  0.]        	[   90. 20097.]
338	228   	[ 10.46666667 966.28333333]  	[  21.62549935 2772.21923431]	[-0.  0.]        	[   91. 18054.]
339	226   	[ 10.09333333 957.55      ]  	[  20.5941243  2933.39131737]	[-0.  0.]        	[   88. 20065.]
340	222   	[ 11.25 995.96]              	[  21.50970711 2804.39720648]	[-0.  0.]        	[   77. 16322.]
341	234   	[  11.59 1267.33]            	[  22.36295821 3533.30481954]	[-0.  0.]        	[   91. 17957.]
342	234   	[ 11.28333333 953.25666667]  	[  22.11974357 3585.189513  ]	[    -0. -27510.]	[   91. 20148.]
343	230   	[ 11.42666667 842.97333333]  	[  22.38834419

...

584	227   	[  12.14       1222.11333333]	[  22.46479616 3363.88521512]	[-0.  0.]        	[   89. 19497.]
585	220   	[  12.35666667 1277.28666667]	[  23.18180584 3299.22792955]	[-0.  0.]        	[   86. 19379.]
586	221   	[  11.14666667 1083.60333333]	[  22.21002376 3008.46758323]	[-0.  0.]        	[   91. 21018.]
587	228   	[  12.06666667 1161.08333333]	[  22.44449945 3077.2497282 ]	[-0.  0.]        	[   92. 20420.]
588	249   	[  11.65666667 1171.39333333]	[  21.73872402 3254.63497266]	[-0.  0.]        	[   77. 20061.]
589	238   	[  14.45666667 1304.08      ]	[  24.58796702 3194.08039874]	[-0.  0.]        	[   89. 21811.]
590	235   	[  13.02       1360.29333333]	[  23.0763862  3404.87404181]	[-0.  0.]        	[   81. 21811.]
591	219   	[  15.10666667 1687.6       ]	[  24.10093959 3937.67311831]	[-0.  0.]        	[   82. 21811.]
592	215   	[  13.08333333 1476.73      ]	[  23.29985098 3744.4241832 ]	[-0.  0.]        	[   91. 19432.]
593	229   	[  10.94333333 1198.13      ]	[  21.41588481

664	241   	[  12.2        1223.20666667]	[  22.99623158 3701.46116157]	[    -0. -18892.]	[   92. 20608.]
665	204   	[  14.38666667 1232.30333333]	[  24.27091172 3449.74273794]	[    -0. -18892.]	[   90. 16798.]
666	240   	[  13.49       1298.13666667]	[  23.84456402 3476.11921708]	[-0.  0.]        	[   90. 21935.]
667	227   	[  12.52666667 1083.74333333]	[  22.60197533 3272.66010519]	[-0.  0.]        	[   87. 21935.]
668	219   	[  12.45666667 1528.41666667]	[  22.27348473 4062.26771845]	[-0.  0.]        	[   87. 21935.]
669	218   	[  11.47       1226.07333333]	[  21.66862017 3455.02374444]	[-0.  0.]        	[   91. 22243.]
670	234   	[  11.53333333 1101.87      ]	[  21.91655894 3132.0340962 ]	[-0.  0.]        	[   89. 19478.]
671	238   	[  12.10333333 1042.15666667]	[  23.11001202 3123.62079625]	[-0.  0.]        	[   84. 19954.]
672	212   	[  12.26       1069.27333333]	[  22.90660167 3125.84362991]	[-0.  0.]        	[   79. 20435.]
673	209   	[  12.87333333 1326.61      ]	[  24.06665651

754	245   	[  13.70666667 1232.74333333]	[  23.70134923 3145.58817777]	[-0.  0.]        	[   92. 18416.]
755	236   	[  12.2  1108.99]            	[  22.36082885 3188.79804784]	[-0.  0.]        	[   92. 20376.]
756	246   	[  12.53333333 1243.30333333]	[  23.43819295 3478.36455986]	[-0.  0.]        	[   92. 20376.]
757	229   	[  13.22666667 1122.33333333]	[  23.2438226  2874.54840782]	[-0.  0.]        	[   88. 19799.]
758	228   	[  12.87333333 1388.28666667]	[  22.47466623 3452.11432475]	[-0.  0.]        	[   80. 19799.]
759	236   	[  12.38333333 1189.28      ]	[  22.97454509 3280.67565525]	[-0.  0.]        	[   90. 20400.]
760	226   	[  11.94333333 1308.15666667]	[  21.71850491 3518.37099126]	[-0.  0.]        	[   88. 21135.]
761	218   	[  13.22       1630.34333333]	[  23.5731118  3948.12869582]	[-0.  0.]        	[   91. 20400.]
762	232   	[  13.35333333 1280.04666667]	[  23.55309935 3515.92281549]	[-0.  0.]        	[   91. 20986.]
763	231   	[  15.02       1422.18666667]	[  24.33515153

838	233   	[ 11.78333333 826.83      ]  	[  22.2536077  3152.34827091]	[    -0. -30975.]	[   85. 20018.]
839	246   	[  11.96       1355.72333333]	[  22.38701409 3571.74040118]	[-0.  0.]        	[   82. 20018.]
840	228   	[  10.72       1215.15666667]	[  21.21983977 3478.96211804]	[-0.  0.]        	[   80. 20205.]
841	226   	[  11.34666667 1294.98      ]	[  21.35586935 3503.80252863]	[-0.  0.]        	[   83. 20359.]
842	220   	[  11.7        1286.60666667]	[  22.02884473 3519.28872245]	[-0.  0.]        	[   82. 19158.]
843	225   	[ 13.25666667 899.28333333]  	[  22.94582291 2475.7723663 ]	[-0.  0.]        	[   88. 18126.]
844	231   	[  13.89333333 1191.23333333]	[  24.11739253 3024.81817397]	[-0.  0.]        	[   90. 18667.]
845	215   	[  15.84333333 1493.30333333]	[  24.8752646  3626.85748429]	[-0.  0.]        	[   88. 20333.]
846	218   	[  15.34333333 1537.54333333]	[  24.24401209 3598.79115743]	[-0.  0.]        	[   88. 20847.]
847	223   	[  13.04333333 1157.23666667]	[  23.69869453

917	233   	[  13.88333333 1145.69666667]	[  23.98269631 3712.4471567 ]	[    -0. -30975.]	[   87. 19039.]
918	215   	[  14.04666667 1182.08      ]	[  22.94190828 3824.21344945]	[    -0. -30975.]	[   74. 20473.]
919	197   	[  15.67333333 1596.62666667]	[  24.91130845 4521.13057991]	[    -0. -30975.]	[   88. 20356.]
920	221   	[  14.7        1541.12333333]	[  24.44483586 3993.18326253]	[   -0. -7251.]  	[   90. 20397.]
921	212   	[  13.34       1355.77333333]	[  23.11574644 3728.72672396]	[   -0. -2191.]  	[   92. 20397.]
922	226   	[  13.48333333 1114.64333333]	[  23.60218328 4015.99230612]	[    -0. -30975.]	[   86. 20356.]
923	233   	[ 12.24       881.23333333]  	[  21.94635277 3608.01330359]	[    -0. -30975.]	[   84. 19398.]
924	225   	[ 12.61666667 803.86333333]  	[  22.04018426 3207.5173709 ]	[    -0. -30975.]	[   92. 19099.]
925	215   	[  14.24333333 1112.06666667]	[  23.50880946 3002.8201193 ]	[   -0. -6828.]  	[   92. 21108.]
926	216   	[  14.62666667 1341.74      ]	[  24.58415388

In [169]:
print(pop)

[Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual({696}), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual({205}), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual({469}), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual({352}), Individual(), Individual(), Individual({285}), Individual(), Individual({632}), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual(), Individual({87}), Individual(), Individual(), Individual(), Individual(),

In [171]:
print(hof)

[Individual(), Individual({383}), Individual({410}), Individual({446}), Individual({230}), Individual({262}), Individual({6}), Individual({563}), Individual({580, 357, 134, 58, 217, 250})]
